# Perspectief 1: Verschillen in milieubeleid verklaren verschillen in landbouwuitstoot

De uitstoot van de landbouwsector verschilt sterk tussen landen. In dit perspectief onderzoeken we hoe beleidskeuzes zoals milieuregels en landbouwsubsidies bijdragen aan die verschillen. We gebruiken internationale data om te analyseren welke landen efficiënt produceren en welke landen vervuilende landbouw blijven stimuleren.


### Argument 1: Strenger milieubeleid leidt tot lagere uitstoot per ton voedsel

Landen verschillen sterk in hoe streng hun milieubeleid is. In sommige landen gelden duidelijke regels voor het gebruik van kunstmest, mestverwerking of methaanuitstoot. Andere landen leggen nauwelijks beperkingen op aan de landbouwsector. De aanname is dat streng milieubeleid boeren aanzet tot efficiëntere en schonere productie.

We gebruiken de Environmental Performance Index (EPI) als indicator voor hoe actief landen hun milieubeleid invullen. Deze score wordt jaarlijks gepubliceerd en kijkt onder andere naar klimaatmaatregelen, landbouwimpact en duurzaamheid van het landgebruik. Door deze scores te koppelen aan uitstootcijfers uit de landbouwsector (in verhouding tot de hoeveelheid geproduceerd voedsel), kunnen we zien of beleid echt effect heeft.

In onderstaande kaart combineren we beide datasets: de kleur laat zien hoe goed een land scoort op milieubeleid, terwijl de grootte van de cirkel aangeeft hoeveel CO₂-uitstoot er vrijkomt per ton voedsel dat wordt geproduceerd.

In [ ]:
import pandas as pd

# Bestanden inladen
emissies = pd.read_csv("docs/FAOSTAT_emissions.csv")
productie = pd.read_csv("docs/FAOSTAT_production.csv")
epi = pd.read_csv("docs/epi2022results05302022.csv")